 # Importar arquivos para leitura

In [1]:
import pandas as pd
import os

In [2]:
base_sip_oem = pd.read_excel(r"C:\Users\OneEngInst\Projetos_python\dashboard\content\base_sip_Concluido_mt.xlsx")
#df_intelbras = pd.read_excel(r"E:\ACRÉSCIMOS PREVENTIVA JUNHO 3ª REMESSA.xlsx")

In [3]:
base_sip_oem = base_sip_oem[['IDSIGFI','NUMOS','NOMECLIENTE','CPF','CODIGOENERGISA','UC','TIPODESIGFI','OBRA','ROTA','EXECUTOR','LATLONCON','TIPOCAUSA','DTCONCLUSAO','ENDERECO','MUNICIPIO']]

In [4]:
print(base_sip_oem.shape)
#print(df_intelbras.shape)

(840, 15)


# Realizar leitura de arquivos  .pdf  dentro das pastas correspondente

In [5]:
diretorio_lote = r"C:\Users\OneEngInst\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\001_INTELBRAS\MT\0.0-PREVENTIVA\2ªTRANCHE\2024\LOTE_04"
#"C:\Users\OneEngInst\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\0.0-PREVENTIVA\1ªTRACHE\2024\LOTE_16\3ªREMESSA"

In [6]:
# listando os arquivos dentro da pasta.
arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

lista_pdf = pd.DataFrame(arquivos_pdf, columns=['NUMOS'])
lista_pdf['NUMOS'] = lista_pdf['NUMOS'].apply(lambda x: x[-22:-10]).astype('int64')
contagem_arquivos = lista_pdf['NUMOS'].unique().shape[0]
dataset = lista_pdf.merge(base_sip_oem, left_on='NUMOS', right_on='NUMOS', how='left')

In [7]:
dataset[['LATLONCON']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   LATLONCON  218 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB


In [8]:
# Rastrear duplicidade e soma-las.
dataset['NUMOS'].duplicated().sum()

0

In [9]:
dataset['NUMOS'].count()

218

In [10]:
def encontrar_duplicados(dataset):
    duplicados = dataset[dataset.duplicated(subset='NUMOS')]
    return duplicados['NUMOS']

# Exemplo de uso
duplicados = encontrar_duplicados(dataset)
print(duplicados)

Series([], Name: NUMOS, dtype: int64)


# Realizar a mesclagem com número de OS entre os dados da base do sistema com a lista de arquivos encontrados na pasta

In [11]:
#dataset[['LATITUDE', 'LONGITUDE','ERRO_1','ERRO_2']] = dataset['LATLONCON'].apply(lambda x: pd.Series(x.split(',')))#,'ERRO_1','ERRO_2'
def split_latlon(x):
    if isinstance(x, str):
        return pd.Series(x.split(','))
    else:
        #Se não for uma string, retorna NaN ou valores padrão, dependendo do que você deseja
        return pd.Series([pd.NaT, pd.NaT])

# Aplica a função split_latlon à coluna 'LATLONCON'
dataset[['LATITUDE', 'LONGITUDE']] = dataset['LATLONCON'].apply(split_latlon)#,'erro_1','erro_2'

# Converta as colunas 'LATITUDE' e 'LONGITUDE' para números, se necessário
#dataset[['LATITUDE', 'LONGITUDE']] = dataset[['LATITUDE', 'LONGITUDE']].apply(pd.to_numeric, errors='coerce')

dataset= dataset[['IDSIGFI','UC','OBRA','NOMECLIENTE','CPF','ENDERECO','MUNICIPIO','DTCONCLUSAO','TIPOCAUSA','ROTA','EXECUTOR','TIPODESIGFI','LATITUDE','LONGITUDE','NUMOS']]#,'NUMOS'
lote = dataset.rename(columns={'IDSIGFI':'CÓDIGO ONE','NOMECLIENTE':'NOME DO CLIENTE','CODIGOENERGISA': 'CÓDIGO','UC': 'NÚM.DA UC',
                               'OBRA':'NÚM.DA OBRA','EXECUTOR':'EQUIPE EXECUTORA',
                              'TIPODESIGFI': 'TIPO/SIGFI','TIPOCAUSA':'PREVENTIVA','DTCONCLUSAO':'DATA ATENDIMENTO'})

replacements1 = {'ONEOM001':'EQ.1','ONEOM002':'EQ.2','ONEOM003':'EQ.3','ONEOM004':'EQ.4','ONEOM005':'EQ.5','ONEOM006':'EQ.6','ONEOM007':'EQ.7',
                 'ONEOM008':'EQ.8','ONEOM009':'EQ.9','ONEOM010':'EQ.10','ONEOM011':'EQ.11','ONEOM012':'EQ.12','ONEOM013':'EQ.13','ONEOM014':'EQ.14',
                 'ONEOM015':'EQ.15'}
lote['EQUIPE EXECUTORA'] = lote['EQUIPE EXECUTORA'].replace(replacements1)

replacements2 = {'MANUT PREVENTIVA 1A':'1ªPREVENTIVA','MANUT PREVENTIVA 2A':'2ªPREVENTIVA','MANUT PREVENTIVA 3A':'3ªPREVENTIVA',
                 'MANUT PREVENTIVA 4A':'4ªPREVENTIVA','MANUT PREVENTIVA 5A':'5ªPREVENTIVA'}
lote['PREVENTIVA'] = lote['PREVENTIVA'].replace(replacements2)

replacements3 = {'QUERÊNCIA':'QUERÊNCIA','FELIZ NATAL':'QUERÊNCIA'}

lote['REGIONAL'] = lote['MUNICIPIO'].replace(replacements3)

lote = lote[['CÓDIGO ONE', 'NÚM.DA UC', 'NÚM.DA OBRA', 'NOME DO CLIENTE', 'CPF',
       'ENDERECO', 'MUNICIPIO','REGIONAL', 'DATA ATENDIMENTO', 'PREVENTIVA', 'ROTA',
       'EQUIPE EXECUTORA', 'TIPO/SIGFI', 'LATITUDE', 'LONGITUDE', 'NUMOS']]

In [12]:
lote

,CÓDIGO ONE,NÚM.DA UC,NÚM.DA OBRA,NOME DO CLIENTE,CPF,ENDERECO,MUNICIPIO,REGIONAL,DATA ATENDIMENTO,PREVENTIVA,ROTA,EQUIPE EXECUTORA,TIPO/SIGFI,LATITUDE,LONGITUDE,NUMOS
0,41029,4398711.0,622302280.0,TAPURM KAIABI,083.994.001-73,TERRA INDÍGENA PARQUE DO XINGU; ALDEIA ILHA GR...,QUERÊNCIA,QUERÊNCIA,2024-05-28 14:18:00,MANUT PREVENTIVA 1A,503.0,ONEMT002,SIGFI80,-11.49792926,-53.45886351,202400001147
1,41007,4399379.0,622302248.0,TAROKO KAIABI,2764430124,ALDEIA ILHA GRANDE ALDEIA ILHA GRANDE,QUERÊNCIA,QUERÊNCIA,2024-07-23 17:18:00,MANUT PREVENTIVA 1A,503.0,ONEMT009,SIGFI80,-11.500608333333332,-53.45872666666667,202400005536
2,40836,4485918.0,622400315.0,KAHAKUAGU KALAPALO,058.847.901-20,ALDEIA KALAPALO S/N ALDEIA KALAPALO,QUERÊNCIA,QUERÊNCIA,2024-06-17 11:14:00,MANUT PREVENTIVA 1A,508.0,ONEMT004,SIGFI80,-12.1623189,-53.25482096,202400002475
3,40835,4485921.0,622400316.0,JULO KALAPALO,044.681.611-67,ALDEIA KALAPALO S/N ALDEIA KALAPALO,QUERÊNCIA,QUERÊNCIA,2024-08-01 13:04:00,MANUT PREVENTIVA 1A,508.0,ONEMT004,SIGFI80,-12.16161839,-53.25530847,202400002384
4,40840,4485956.0,622400258.0,DIDA KOHÔI KALAPALO,037.886.551-00,ALDEIA KALAPALO S/N ALDEIA KALAPALO,QUERÊNCIA,QUERÊNCIA,2024-06-05 11:45:00,MANUT PREVENTIVA 1A,508.0,ONEMT005,SIGFI80,-12.163561666666665,-53.25473,202400005544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,60541,4603039.0,622401268.0,BRUNA KUIKURO,10364645121,TERRA INDÍGENA PARQUE DO XINGU; POLI INDIGENA ...,GAÚCHA DO NORTE,GAÚCHA DO NORTE,2024-08-29 08:54:00,MANUT PREVENTIVA 1A,509.0,ONEMT006,SIGFI45,-12.199616666666667,-53.376999999999995,202400009439
214,61273,4604590.0,62240139.0,KAYADU JURUNA,047.073.281-48,TERRA INDÍGENA PARQUE DO XINGU; ALDEIA AWAYA; ...,SÃO FÉLIX DO ARAGUAIA,SÃO FÉLIX DO ARAGUAIA,2024-08-26 12:07:00,MANUT PREVENTIVA 1A,502.0,ONEMT002,SIGFI80,-10.936826666666665,-53.061863333333335,202400010247
215,61275,4604739.0,622401391.0,DURIMADÊ JURUNA,043.943.811-00,TERRA INDÍGENA PARQUE DO XINGU; ALDEIA AWAYA; ...,SÃO FÉLIX DO ARAGUAIA,SÃO FÉLIX DO ARAGUAIA,2024-08-26 10:23:00,MANUT PREVENTIVA 1A,502.0,ONEMT002,SIGFI45,-10.93694,-53.061776666666674,202400010254
216,60799,4608460.0,62240199.0,UNIDADE BÁSICA DE SAÚDE POLO LEONARDO,00.394.544/0045-04,TERRA INDÍGENA PARQUE DO XINGU; POLO INDÍGENA ...,GAÚCHA DO NORTE,GAÚCHA DO NORTE,2024-09-06 08:08:00,MANUT PREVENTIVA 1A,509.0,ONEMT006,SIGFI160,-12.200408333333334,-53.377165,202400012797


In [14]:
# Salvando a planilha( .xlsx)
dataframe_excel = pd.ExcelWriter(r"C:\Users\OneEngInst\Projetos_python\notebook\OEM\output\2ºtrache_intelbras_mt_preventiva_lote4_remessa01_2024.xlsx")
lote.to_excel(dataframe_excel, index=False)
dataframe_excel.close()